<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/LDA_classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [0]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

#Load data

In [34]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
public_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
PA_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_labels = df_train.Histology

In [0]:
PA_labels = df_test.Histology

#Train Test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(public_data, public_labels, test_size=13, stratify=public_labels, random_state=1)

In [46]:
X_train.shape

(118, 107)

#Vettorizzare i label

In [0]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_labels_encoded = encoder.fit_transform(y_train)
test_labels_encoded = encoder.transform(y_test)

#Z score dei dati

In [0]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#train_data_stand = scaler.fit_transform(X_train)
#test_data_stand = scaler.transform(X_test)

#LDA

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [0]:
lda = LinearDiscriminantAnalysis(n_components=2)

In [0]:
train_data_stand_lda = lda.fit_transform(X_train, train_labels_encoded)

In [52]:
train_data_stand_lda.shape

(118, 2)

In [0]:
test_data_stand_lda = lda.transform(X_test)

#Z score dei dati dopo LDA

In [0]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#train_data_stand = scaler.fit_transform(train_data_stand_lda)
#test_data_stand = scaler.transform(train_data_stand_lda)

#LDA classification

In [55]:
lda.score(X_test, test_labels_encoded)

0.38461538461538464

#Cross_validation (stratified) results

In [0]:
from sklearn.model_selection import StratifiedKFold

In [57]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(X_train, train_labels_encoded)

3

In [0]:
X_train_np = X_train.to_numpy()

In [59]:
print('LDA results: ')

results=[]
for train_index, val_index in skf.split(X_train_np, train_labels_encoded):
 
  partial_train_data = np.array([X_train_np[i] for i in train_index])
  partial_train_targets = np.array([train_labels_encoded[i] for i in train_index])
  
  val_data = np.array([X_train_np[i] for i in val_index])
  val_targets = np.array([train_labels_encoded[i] for i in val_index])

  lda.fit(partial_train_data, partial_train_targets)
  outVal = lda.score(val_data, val_targets)
  results.append(outVal)
print('Score: ' + str(results))

LDA results: 
Score: [0.3, 0.3333333333333333, 0.4358974358974359]


In [60]:
results

[0.3, 0.3333333333333333, 0.4358974358974359]

In [0]:
a=np.array(results)

In [62]:
a.mean()

0.3564102564102564